In [12]:
import numpy as np
from pathlib import Path
import SimpleITK as sitk
import pandas as pd
from glob import glob
from tqdm import tqdm

In [7]:
paths=sorted(glob(f'/home/kakeya/ssd/strange/*/SE2.nii.gz'))

In [9]:
for path in paths:
    img = sitk.ReadImage(str(path))
    # boolを渡して、interpolatorを変更している
    img_array = sitk.GetArrayFromImage(img)
    print(path,img_array.shape)

/home/kakeya/ssd/strange/00021/SE2.nii.gz (59, 512, 512)
/home/kakeya/ssd/strange/00074/SE2.nii.gz (58, 512, 512)
/home/kakeya/ssd/strange/00140/SE2.nii.gz (58, 512, 512)
/home/kakeya/ssd/strange/00154/SE2.nii.gz (57, 512, 512)
/home/kakeya/ssd/strange/00159/SE2.nii.gz (58, 512, 512)


In [25]:
def PadCenter(img: np.array, padded_size=(60,512,512)):
    """
    zyxに並んでいる必要がある。
    Spacing幅を変えたときにxyをpadding
    """
    if img.shape[1] > padded_size[1]:
        raise ValueError('this img is lager than padded_size')
    if img.shape[0] > padded_size[0]:
        raise ValueError('this img is lager than padded_size')
    if img.shape[1] != img.shape[2]:
        raise ValueError('this img is not square.')

    hw_pad_size = padded_size[1] - img.shape[1]
    d_pad_size = padded_size[0] - img.shape[0]
    print(hw_pad_size,d_pad_size)

    r_hw_pad = hw_pad_size // 2
    l_hw_pad = int(np.ceil(hw_pad_size / 2))
    t_d_pad = d_pad_size // 2
    u_d_pad = int(np.ceil(d_pad_size / 2))

    return np.pad(img, ((t_d_pad, u_d_pad), (r_hw_pad, l_hw_pad), (r_hw_pad, l_hw_pad)), 'minimum')
              
def save_niigz(save_path: str, array: np.array):
    # (z,x,y)の方向になるようにする。
    DIRECTION = (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
    array = array.astype(np.int16)
    save_image = sitk.GetImageFromArray(array)
    save_image.SetSpacing((0.7, 0.7, 2.0))
    save_image.SetDirection(DIRECTION)
    if not save_path.parent.is_dir():
        save_path.parent.mkdir()
        print(save_path)
    sitk.WriteImage(save_image, str(save_path), True)

In [31]:
from pathlib import Path
paths=sorted(glob(f'/home/kakeya/ssd/strange/*/*.nii.gz'))
for path in paths:
    path=Path(path)
    img = sitk.ReadImage(str(path))
    # boolを渡して、interpolatorを変更している
    img_array = sitk.GetArrayFromImage(img)
    slices=img_array.shape[0]
    if slices<60:
        print(path,img_array.shape)
    img_array=PadCenter(img_array, padded_size=(60,512,512))    
    print(img_array.shape)
    save_niigz(path,img_array)
    

/home/kakeya/ssd/strange/00021/CCRCC.nii.gz (59, 512, 512)
0 1
(60, 512, 512)
/home/kakeya/ssd/strange/00021/SE2.nii.gz (59, 512, 512)
0 1
(60, 512, 512)
/home/kakeya/ssd/strange/00021/SE3.nii.gz (59, 512, 512)
0 1
(60, 512, 512)
/home/kakeya/ssd/strange/00021/kidney.nii.gz (59, 512, 512)
0 1
(60, 512, 512)
/home/kakeya/ssd/strange/00074/CCRCC.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00074/SE2.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00074/SE3.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00074/cyst.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00074/kidney.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00140/CCRCC.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00140/SE2.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00140/SE3.nii.gz (58, 512, 512)
0 2
(60, 512, 512)
/home/kakeya/ssd/strange/00140/kidney.nii.gz (58, 512, 512)
0 2
(60, 512, 5

In [18]:
npys=glob('/home/kakeya/ssd/deeds_data/*/tumor_48x48x16_/patch_image_*.npy')
for i,path in enumerate(tqdm(npys)):
    img_array = np.load(str(path))
    # boolを渡して、interpolatorを変更している
    if img_array.shape!=(48, 48, 16, 2):
        print(path,img_array.shape)

    

100%|██████████| 188161/188161 [03:11<00:00, 982.85it/s] 


In [21]:
npys=glob('/home/kakeya/ssd/deeds_data/*/tumor_48x48x16_/patch_no_*.npy')
for i,path in enumerate(tqdm(npys)):
    img_array = np.load(str(path))
    # boolを渡して、interpolatorを変更している
    if img_array.shape!=(48, 48, 16):
        print(path,img_array.shape)

100%|██████████| 188161/188161 [02:09<00:00, 1450.01it/s]
